### Let's deploy an application that explores rate limiting

In [11]:
# get to the right directory
cd ~/projects/spring-cloud-gateway-workshop

In [ ]:
#check what services are setup
cf services

In [ ]:
#check the market place to see plan options are available for Spring Cloud Gateway
cf marketplace

In [ ]:
#see which apps are currently deployed
cf apps

In [ ]:
#create a new service instance of the gateway
cf create-service p.gateway standard ratelimit-gateway -c '{"host": "ratelimit-gateway"}'


In [16]:
#push our rate limiting app
cd ratelimiting
cf push

Pushing from manifest to org scg-workshop / space demo as brian...
Using manifest file /Users/brianbyers/projects/spring-cloud-gateway-workshop/ratelimiting/manifest.yml
Getting app info...
Creating app with these attributes...
+ name:       ratelimiting
  path:       /Users/brianbyers/projects/spring-cloud-gateway-workshop/ratelimiting
+ command:    node index.js
+ memory:     128M
  env:
+   OPTIMIZE_MEMORY
  routes:
+   ratelimiting.apps.internal
+   ratelimiting.cfapps.pcf.cloud

Creating app ratelimiting...
Mapping routes...
Comparing local files to remote cache...
Packaging files to upload...
Uploading files...
 609.25 KiB / 609.25 KiB [==========================================] 100.00% 1s

Waiting for API to complete processing files...

Staging app and tracing logs...
   Downloaded ruby_buildpack
   Downloaded java_buildpack_offline
   Downloaded nodejs_buildpack
   Downloaded binary_buildpack
   Downloaded staticfile_buildpack
   Downloaded nginx_buildpack
   Downloaded go_bu

In [17]:
#bind our app to the rate limiting service
cf bind-service ratelimiting ratelimit-gateway -c ./scg-config.json


Binding service ratelimit-gateway to app ratelimiting in org scg-workshop / space demo as brian...
OK

TIP: Use 'cf restage ratelimiting' to ensure your env variable changes take effect


In [18]:
#test out our no limit
# https://ratelimit-gateway.cfapps.pcf.cloud/ratelimiting/nolimit
for i in {1..20}; do sleep .1; curl -o -I -L -s -w "%{http_code}" https://ratelimit-gateway.cfapps.pcf.cloud/ratelimiting/nolimit ;echo '\n';done

200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n


In [19]:
# test out 1 per second by calling 4 times per second
# https://ratelimit-gateway.cfapps.pcf.cloud/ratelimiting/1persec
# should get a successful response 1 in 4 times, may come across as 1 in 3 due to timing
for i in {1..20}; do sleep .25; curl -o -I -L -s -w "%{http_code}" https://ratelimit-gateway.cfapps.pcf.cloud/ratelimiting/1persec ;echo '\n';done

200\n
429\n
429\n
200\n
429\n
429\n
200\n
429\n
429\n
200\n
429\n
429\n
200\n
429\n
429\n
200\n
429\n
429\n
200\n
429\n


In [20]:
#test out 1 per second by calling 1 time per second
# https://ratelimit-gateway.cfapps.pcf.cloud/ratelimiting/1persec
# should be all 200s
for i in {1..10}; do sleep 1; curl -o -I -L -s -w "%{http_code}" https://ratelimit-gateway.cfapps.pcf.cloud/ratelimiting/1persec ;echo '\n';done

200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n


In [21]:
#test out 10 per minute by calling 1 time per second
# https://ratelimit-gateway.cfapps.pcf.cloud/ratelimiting/10permin
for i in {1..20}; do sleep 1; curl -o -I -L -s -w "%{http_code}" https://ratelimit-gateway.cfapps.pcf.cloud/ratelimiting/10permin ;echo '\n';done

200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
200\n
429\n
429\n
429\n
429\n
200\n
429\n
429\n
429\n
